Desinstalamos las versiones existentes de **tensorflow** y **keras**, y si exist euna versión previa de la librería  **mrcnn** e instalamos de nuevo en las versiones necesarias.

In [ ]:
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow-gpu
!pip uninstall -y keras
!pip uninstall -y mrcnn

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
Uninstalling Keras-2.4.3:
  Successfully uninstalled Keras-2.4.3


In [ ]:
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.2.5
!pip install mrcnn

     |████████████████████████████████| 377.1MB 41kB/s 
     |████████████████████████████████| 3.2MB 51.6MB/s 
     |████████████████████████████████| 491kB 53.7MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
     |████████████████████████████████| 337kB 8.2MB/s 
ERROR: fancyimpute 0.4.3 requires tensorflow, which is not installed.
     |████████████████████████████████| 61kB 4.8MB/s 
  Created wheel for mrcnn: filename=mrcnn-0.2-cp37-none-any.whl size=54930 sha256=f3683f594defacbeb71c3dc3228752bebcd5b50e2d85d7e1474626a50a49858c
  Stored in directory: /root/.cache/pip/wheels/11/ed/28/e550ddc897c04c336b923eae4eb35c9aae993d20ce39d9cc40
Successfully built mrcnn


**CarConfig** ,   **CarDataset**, are the classes that load the dataset of category detector CAR.

In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
from mrcnn.config import Config
from mrcnn import model as modellib, utils


category_name="car"
Class_names=["car"]
# Class_names=["car","bike","motorbike"]
############################################################
#  Configurations
############################################################

class CarConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "car"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + len(Class_names)  # Background + car

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


############################################################
#  Dataset
############################################################
class CarDataset(utils.Dataset):

    def load_car(self, dataset_dir, subset):
        """Load a subset of the Car dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        for cls in range(len(Class_names)):
          # Add classes. We have only one class to add.
          self.add_class(category_name, cls+1, Class_names[cls])

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, "via_export_json.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']] 

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                category_name,
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a car dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "car":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        print(info)
        if info["source"] == "car":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [ ]:

def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CarDataset()
    dataset_train.load_car(dataset_folder, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CarDataset()
    dataset_val.load_car(dataset_folder, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=10,
                layers='heads')



In [ ]:
############################################################
#  load dataset from drive
############################################################
from google.colab import drive
from pathlib import Path

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
Path.ls = lambda x: list(x.iterdir())
path = Path('/content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/')
print(path.ls())
# Root directory of the project
ROOT_DIR = os.path.abspath("./")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(path, "weights/mask_rcnn_coco.h5")
weights_db="coco"  #Tells if db weights are COCO to drop some layers for new classes.

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(path, "logs")

[PosixPath('/content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/dataset'), PosixPath('/content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/weights'), PosixPath('/content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/logs (1)'), PosixPath('/content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/logs'), PosixPath('/content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/Untitled0.ipynb')]


In [ ]:


weights_file=path/'weights/mask_rcnn_coco.h5'
dataset_folder=path/'dataset'
logs_folder=path/'logs'

print("Weights: ",weights_file)
print("Dataset: ", dataset_folder)
print("Logs: ", logs_folder)

config = CarConfig()
  
model = modellib.MaskRCNN(mode="training", config=config, model_dir=logs_folder)


# Load weights
print("Loading weights ", COCO_WEIGHTS_PATH)
# Exclude the last layers because they require a matching
# number of classes
if weights_db=="coco":
  model.load_weights(str(COCO_WEIGHTS_PATH), by_name=True, exclude=[
              "mrcnn_class_logits", "mrcnn_bbox_fc",
              "mrcnn_bbox", "mrcnn_mask"])
else:
  model.load_weights(str(COCO_WEIGHTS_PATH), by_name=True)

train(model)

Weights:  /content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/weights/mask_rcnn_coco.h5
Dataset:  /content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/dataset
Logs:  /content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/logs
Loading weights  /content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/weights/mask_rcnn_coco.h5
Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /content/gdrive/MyDrive/PROYECTOS_IT/PAVIMENTO/MaskRCNN/logs/car20210308T2046/mask_rcnn_car_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistribu

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multiprocessing=True` and multiple worker

Epoch 1/10
100/100 [==============================] - 145s 1s/step - loss: 1.3467 - rpn_class_loss: 0.0224 - rpn_bbox_loss: 0.0901 - mrcnn_class_loss: 0.2992 - mrcnn_bbox_loss: 0.3342 - mrcnn_mask_loss: 0.6007 - val_loss: 0.7802 - val_rpn_class_loss: 0.0135 - val_rpn_bbox_loss: 0.0721 - val_mrcnn_class_loss: 0.1409 - val_mrcnn_bbox_loss: 0.1418 - val_mrcnn_mask_loss: 0.4119


Process ForkPoolWorker-15:
Process ForkPoolWorker-16:
Traceback (most recent call last):
Process ForkPoolWorker-14:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-13:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/pro

KeyboardInterrupt: ignored